In [2]:
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
from sklearn import model_selection
import numpy as np

In [17]:
df = pd.read_csv('sea.csv', names = ['feature1','feature2','feature3','label'])

In [18]:
df

,feature1,feature2,feature3,label
0,6.677259,5.152133,2.982455,0
1,9.874437,8.817701,4.786266,1
2,7.118725,2.990575,1.964403,0
3,6.128244,8.449696,2.604408,1
4,7.592623,0.859845,5.763160,0
...,...,...,...,...
59995,4.760386,7.501301,5.680291,1
59996,7.873042,7.951943,1.213854,1
59997,8.430418,9.531408,1.034550,0
59998,6.438425,8.759620,6.009597,1


In [36]:
df[(df.sum_1_2<= 7) & (df.label == 1)]

,feature1,feature2,feature3,label,sum_1_2
16,0.278338,4.522238,5.566390,1,4.800576
40,4.441528,2.296754,5.816807,1,6.738282
43,1.300799,3.137467,7.350058,1,4.438266
50,1.461976,4.200469,3.934796,1,5.662445
57,3.182621,3.611762,9.662165,1,6.794383
...,...,...,...,...,...
59915,2.266173,4.500879,6.974298,1,6.767052
59924,0.514113,2.141795,2.882903,1,2.655908
59980,0.022763,3.428859,8.098660,1,3.451622
59981,0.163419,4.537891,8.206822,1,4.701310


In [20]:
df['sum_1_2'] = df['feature1'] + df['feature2'] 

In [21]:
df

,feature1,feature2,feature3,label,sum_1_2
0,6.677259,5.152133,2.982455,0,11.829392
1,9.874437,8.817701,4.786266,1,18.692138
2,7.118725,2.990575,1.964403,0,10.109300
3,6.128244,8.449696,2.604408,1,14.577940
4,7.592623,0.859845,5.763160,0,8.452468
...,...,...,...,...,...
59995,4.760386,7.501301,5.680291,1,12.261687
59996,7.873042,7.951943,1.213854,1,15.824985
59997,8.430418,9.531408,1.034550,0,17.961826
59998,6.438425,8.759620,6.009597,1,15.198045


Take the first 15000 and train the model on it

In [22]:

df1  = df[:15000]

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [31]:
df1[df1.sum_1_2 > 9.5].label.value_counts()

1    6269
0    2036
Name: label, dtype: int64

In [32]:
df2 = df[15000:30000]

In [33]:
df2[df2.sum_1_2 > 9].label.value_counts()

1    6119
0    2817
Name: label, dtype: int64

In [25]:
df2[df2['label'] == 0].sum_1_2.mean()

8.488388368495967

In [14]:
df3 = df[30000:45000]

In [26]:
df3[df3['label'] == 0].sum_1_2.mean()

8.290075725759282

In [16]:
df3[df3['label'] == 1].sum_1_2.max()

19.905741

In [5]:
df1.label.value_counts()

1    9683
0    5317
Name: label, dtype: int64

In [6]:

X = df1[['feature1', 'feature2', 'feature3']].values
y = df1['label'].values


In [7]:
# split data into train and test sets
seed = 7
test_size = 0.33
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=seed)
scoring = ['accuracy', 'precision_weighted', 'recall_weighted', 'f1_weighted', 'roc_auc']
# fit model no training data
model = XGBClassifier(n_estimators = 500, max_depth=10)
model.fit(X_train, y_train)
print(model)

/usr/local/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[23:00:33] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=10,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=500, n_jobs=8, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)


In [8]:
kfold = model_selection.KFold(n_splits=5, shuffle=True, random_state=90210)
cv_results = model_selection.cross_validate(model, X_train, y_train, cv=kfold, scoring=scoring)
# clf = model.fit(X_train, y_train,  eval_metric = 'logloss')

[23:00:44] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/usr/local/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[23:00:51] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/usr/local/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[23:00:57] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/usr/local/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[23:01:03] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/usr/local/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[23:01:13] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [9]:
cv_results

{'fit_time': array([7.13320875, 6.04660058, 5.86430216, 9.28708029, 5.20937109]),
 'score_time': array([0.10943937, 0.05604529, 0.04121876, 0.04715586, 0.05700636]),
 'test_accuracy': array([0.85223881, 0.86318408, 0.8681592 , 0.8681592 , 0.85024876]),
 'test_precision_weighted': array([0.85064445, 0.86186032, 0.86707593, 0.86678074, 0.84902774]),
 'test_recall_weighted': array([0.85223881, 0.86318408, 0.8681592 , 0.8681592 , 0.85024876]),
 'test_f1_weighted': array([0.85084073, 0.86170325, 0.86690771, 0.8667476 , 0.84827401]),
 'test_roc_auc': array([0.85529158, 0.85587389, 0.87697703, 0.87996717, 0.85723713])}

In [10]:
# make predictions for test data
y_pred = model.predict(X_test)
# predictions = [round(value) for value in y_pred]
predictions = model.predict_proba(X_test)

In [11]:
# evaluate predictions
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 86.99%


Use PSI to check model's current decile distribution

Get the probabilities of class 0 

In [12]:
predictions

array([[0.83737445, 0.16262552],
       [0.007029  , 0.992971  ],
       [0.04399091, 0.9560091 ],
       ...,
       [0.02260154, 0.97739846],
       [0.5994914 , 0.40050858],
       [0.08273053, 0.91726947]], dtype=float32)

In [13]:
predictions.shape

(4950, 2)

The first column in the matrix represents the propbability of being class 0

In [14]:
prob_0 = predictions[:,0]

In [15]:
df_prob_0 = pd.DataFrame(prob_0, columns = ['prob_0'])

In [16]:
df_prob_0

,prob_0
0,0.837374
1,0.007029
2,0.043991
3,0.004101
4,0.000281
...,...
4945,0.001342
4946,0.014207
4947,0.022602
4948,0.599491


In [17]:
conditions = [(df_prob_0.prob_0 >= 0.9),
              ((df_prob_0.prob_0 < 0.9) & (df_prob_0.prob_0 >= 0.8)),
              ((df_prob_0.prob_0 < 0.8) & (df_prob_0.prob_0 >= 0.7)),
              ((df_prob_0.prob_0 < 0.7) & (df_prob_0.prob_0 >= 0.6)),
              ((df_prob_0.prob_0 < 0.6) & (df_prob_0.prob_0 >= 0.5)),
              ((df_prob_0.prob_0 < 0.5) & (df_prob_0.prob_0 >= 0.4)),
              ((df_prob_0.prob_0 < 0.4) & (df_prob_0.prob_0 >= 0.3)),
              ((df_prob_0.prob_0 < 0.3) & (df_prob_0.prob_0 >= 0.2)),
              ((df_prob_0.prob_0 < 0.2) & (df_prob_0.prob_0 >= 0.1)),
              (df_prob_0.prob_0 < 0.1)]

choices = np.arange(1,11)

In [18]:
df_prob_0['decile'] = np.select(conditions, choices, default = 'none')

df_prob_0['decile'] = df_prob_0['decile'].astype(int)

df_prob_0

,prob_0,decile
0,0.837374,2
1,0.007029,10
2,0.043991,10
3,0.004101,10
4,0.000281,10
...,...,...
4945,0.001342,10
4946,0.014207,10
4947,0.022602,10
4948,0.599491,5


Count number of values in each decile

In [19]:
traininig_distribution = df_prob_0.groupby('decile')[['prob_0']].count().reset_index().sort_values(by = 'decile')
traininig_distribution = traininig_distribution.rename(columns = {'prob_0' : 'train_dist'})

In [20]:
traininig_distribution['train_dist_perc'] = np.round(100*traininig_distribution['train_dist']/traininig_distribution['train_dist'].sum(),1)

In [21]:
traininig_distribution

,decile,train_dist,train_dist_perc
0,1,1242,25.1
1,2,149,3.0
2,3,81,1.6
3,4,61,1.2
4,5,74,1.5
5,6,66,1.3
6,7,85,1.7
7,8,124,2.5
8,9,287,5.8
9,10,2781,56.2


Now keeping the model same make predictions on the next 15000 of SEA dataset and compare the decile distribution

In [22]:
df2=df[15000:30000]

In [23]:
df2.label.value_counts()

1    8677
0    6323
Name: label, dtype: int64

In [24]:
# df2['feature1'] = df2['feature1']+3
# df2['feature2'] = df2['feature2']+1
df2['feature3'] = df2['feature3']+3

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


take the same number of samples we had in X test (4950), although this isn't required because we take % of samples in the range.

In [25]:

X2 = df2[['feature1', 'feature2', 'feature3']].values
y2 = df2['label'].values

In [26]:
seed = 7
test_size = 0.33
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size=test_size, random_state=seed)

In [27]:
# make predictions for test data
# y2_pred = model.predict(X2_test)
y2_pred = model.predict(X2)
# predictions = [round(value) for value in y_pred]
# predictions = model.predict_proba(X2_test)
predictions2 = model.predict_proba(X2)

In [28]:
# evaluate predictions
# accuracy = accuracy_score(y2_test, y2_pred)
accuracy = accuracy_score(y2, y2_pred)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 65.34%


Accuracy reduced for the changed concept

Check PSI

In [29]:
predictions2

array([[6.6155791e-03, 9.9338442e-01],
       [9.9917626e-01, 8.2374591e-04],
       [3.0110061e-02, 9.6988994e-01],
       ...,
       [1.8010736e-02, 9.8198926e-01],
       [2.8980315e-02, 9.7101969e-01],
       [6.5368414e-04, 9.9934632e-01]], dtype=float32)

In [30]:
prob_0_2 = predictions2[:,0]

In [31]:
df_prob_0_2 = pd.DataFrame(prob_0_2, columns = ['prob_0'])

In [32]:
df_prob_0_2

,prob_0
0,0.006616
1,0.999176
2,0.030110
3,0.000917
4,0.002537
...,...
14995,0.036128
14996,0.062892
14997,0.018011
14998,0.028980


In [33]:
conditions = [(df_prob_0_2.prob_0 >= 0.9),
              ((df_prob_0_2.prob_0 < 0.9) & (df_prob_0_2.prob_0 >= 0.8)),
              ((df_prob_0_2.prob_0 < 0.8) & (df_prob_0_2.prob_0 >= 0.7)),
              ((df_prob_0_2.prob_0 < 0.7) & (df_prob_0_2.prob_0 >= 0.6)),
              ((df_prob_0_2.prob_0 < 0.6) & (df_prob_0_2.prob_0 >= 0.5)),
              ((df_prob_0_2.prob_0 < 0.5) & (df_prob_0_2.prob_0 >= 0.4)),
              ((df_prob_0_2.prob_0 < 0.4) & (df_prob_0_2.prob_0 >= 0.3)),
              ((df_prob_0_2.prob_0 < 0.3) & (df_prob_0_2.prob_0 >= 0.2)),
              ((df_prob_0_2.prob_0 < 0.2) & (df_prob_0_2.prob_0 >= 0.1)),
              (df_prob_0_2.prob_0 < 0.1)]

choices = np.arange(1,11)

In [34]:
df_prob_0_2['decile'] = np.select(conditions, choices, default = 'none')

df_prob_0_2['decile'] = df_prob_0_2['decile'].astype(int)

df_prob_0_2

,prob_0,decile
0,0.006616,10
1,0.999176,1
2,0.030110,10
3,0.000917,10
4,0.002537,10
...,...,...
14995,0.036128,10
14996,0.062892,10
14997,0.018011,10
14998,0.028980,10


In [35]:
serving_distribution = df_prob_0_2.groupby('decile')[['prob_0']].count().reset_index().sort_values(by = 'decile')
serving_distribution = serving_distribution.rename(columns = {'prob_0' : 'serv_dist'})

In [36]:
serving_distribution['serv_dist_perc'] = np.round(100*serving_distribution['serv_dist']/serving_distribution['serv_dist'].sum(),1)

In [37]:
serving_distribution

,decile,serv_dist,serv_dist_perc
0,1,1395,9.3
1,2,267,1.8
2,3,205,1.4
3,4,215,1.4
4,5,236,1.6
5,6,289,1.9
6,7,411,2.7
7,8,623,4.2
8,9,1320,8.8
9,10,10039,66.9


In [38]:
merged = pd.merge(traininig_distribution, serving_distribution, on='decile', how ='inner')

In [39]:
merged

,decile,train_dist,train_dist_perc,serv_dist,serv_dist_perc
0,1,1242,25.1,1395,9.3
1,2,149,3.0,267,1.8
2,3,81,1.6,205,1.4
3,4,61,1.2,215,1.4
4,5,74,1.5,236,1.6
5,6,66,1.3,289,1.9
6,7,85,1.7,411,2.7
7,8,124,2.5,623,4.2
8,9,287,5.8,1320,8.8
9,10,2781,56.2,10039,66.9


In [40]:
merged['s-t'] = merged['serv_dist_perc'] - merged['train_dist_perc']

In [41]:
merged['ln(s/t)'] = np.log(merged['serv_dist_perc']/merged['train_dist_perc'])

In [42]:
merged['PSI'] = merged['ln(s/t)'] * (merged['s-t']/100)

In [43]:
merged.PSI.sum()

0.21051833266665795

In [44]:
merged

,decile,train_dist,train_dist_perc,serv_dist,serv_dist_perc,s-t,ln(s/t),PSI
0,1,1242,25.1,1395,9.3,-15.8,-0.992853,0.156871
1,2,149,3.0,267,1.8,-1.2,-0.510826,0.006130
2,3,81,1.6,205,1.4,-0.2,-0.133531,0.000267
3,4,61,1.2,215,1.4,0.2,0.154151,0.000308
4,5,74,1.5,236,1.6,0.1,0.064539,0.000065
5,6,66,1.3,289,1.9,0.6,0.379490,0.002277
6,7,85,1.7,411,2.7,1.0,0.462624,0.004626
7,8,124,2.5,623,4.2,1.7,0.518794,0.008819
8,9,287,5.8,1320,8.8,3.0,0.416894,0.012507
9,10,2781,56.2,10039,66.9,10.7,0.174282,0.018648


In [45]:
df3=df[30000:45000]

##take the same number of samples we had in X test (4950), although this isn't required because we take % of samples in the range.


X3 = df3[['feature1', 'feature2', 'feature3']].values
y3 = df3['label'].values

seed = 7
test_size = 0.33
X3_train, X3_test, y3_train, y3_test = train_test_split(X3, y3, test_size=test_size, random_state=seed)

# make predictions for test data
y3_pred = model.predict(X3_test)
# predictions = [round(value) for value in y_pred]
predictions = model.predict_proba(X3_test)

# evaluate predictions
accuracy = accuracy_score(y3_test, y3_pred)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

#Accuracy reduced for the changed concept



Accuracy: 81.94%


In [46]:
#Check PSI

predictions

prob_0_3 = predictions[:,0]

df_prob_0_3 = pd.DataFrame(prob_0_3, columns = ['prob_0'])

df_prob_0_3

conditions = [(df_prob_0_3.prob_0 >= 0.9),
              ((df_prob_0_3.prob_0 < 0.9) & (df_prob_0_3.prob_0 >= 0.8)),
              ((df_prob_0_3.prob_0 < 0.8) & (df_prob_0_3.prob_0 >= 0.7)),
              ((df_prob_0_3.prob_0 < 0.7) & (df_prob_0_3.prob_0 >= 0.6)),
              ((df_prob_0_3.prob_0 < 0.6) & (df_prob_0_3.prob_0 >= 0.5)),
              ((df_prob_0_3.prob_0 < 0.5) & (df_prob_0_3.prob_0 >= 0.4)),
              ((df_prob_0_3.prob_0 < 0.4) & (df_prob_0_3.prob_0 >= 0.3)),
              ((df_prob_0_3.prob_0 < 0.3) & (df_prob_0_3.prob_0 >= 0.2)),
              ((df_prob_0_3.prob_0 < 0.2) & (df_prob_0_3.prob_0 >= 0.1)),
              (df_prob_0_3.prob_0 < 0.1)]

choices = np.arange(1,11)

df_prob_0_3['decile'] = np.select(conditions, choices, default = 'none')

df_prob_0_3['decile'] = df_prob_0_3['decile'].astype(int)

df_prob_0_3

serving_distribution = df_prob_0_3.groupby('decile')[['prob_0']].count().reset_index().sort_values(by = 'decile')
serving_distribution = serving_distribution.rename(columns = {'prob_0' : 'serv_dist'})

serving_distribution['serv_dist_perc'] = np.round(100*serving_distribution['serv_dist']/serving_distribution['serv_dist'].sum(),1)

merged = pd.merge(traininig_distribution, serving_distribution, on='decile', how ='inner')

merged

merged['s-t'] = merged['serv_dist_perc'] - merged['train_dist_perc']

merged['ln(s/t)'] = np.log(merged['serv_dist_perc']/merged['train_dist_perc'])

merged['PSI'] = merged['ln(s/t)'] * (merged['s-t']/100)



In [47]:
merged.PSI.sum()


0.0025026061087326714